In [18]:
!pip install spacy

In [30]:
from spacy.displacy.render import EntityRenderer
from IPython.core.display import display, HTML
import spacy
import pandas as pd
nlp = spacy.load('en_core_web_sm')

In [21]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

In [22]:
# colors for additional part of speech tags we want to visualize
options = {
    'colors': {'COMPOUND': '#FE6BFE', 'PROPN': '#18CFE6', 'NOUN': '#18CFE6', 'NP': '#1EECA6', 'ENTITY': '#FF8800'}
}

In [23]:
pd.set_option('display.max_rows', 1000) # edit how jupyter will render our pandas dataframes
pd.options.mode.chained_assignment = None # prevent warning about working on a copy of a dataframe

In [24]:
df = pd.read_csv("data1.csv")

mini_df = df[:200]
mini_df.index = pd.RangeIndex(len(mini_df.index))

# comment this out to run on full dataset
df = mini_df

In [25]:
df

,event_name,event_date,event_location,event_location_url,event_time,event_description
0,FebFest 2020 Winter Park in Boucher Park,01-Mar-20,Boucher Park,https://www.visitkingston.ca/venue/boucher-park/,8:00 AM,Bundle up the little kids and take them to Boucher Park on Clarence Street just around the corner from Springer Market Square and let the kids enjoy the ice slides and ice sculptures. Boucher Park will be open for the whole month of February (weather permitting) in the heart of downtown Kingston ...
1,Free Outdoor Public Skating in Springer Market Square,01-Mar-20,Market Square,https://www.visitkingston.ca/venue/market-square/,8:00 AM,"Spring Market Square - come and enjoy a lovely, outdoor, skate to music in the historical downtown of Kingston. This public skate is free to the public and makes for a great family outing. Hours of operation are from 8 am to 10 pm Monday to Sunday ...."
2,The 2020 Tim Hortons Brier,01-Mar-20,Leon's Centre,https://www.visitkingston.ca/venue/leons-centre/,9:00 AM,"IT'S HERE!! The 2020 Tim Hortons Brier, presented by AGI, will be played Feb. 29-March 8 at Leon’s Centre in downtown Kingston.The Tim Hortons Brier is probably the truest of any Canadian sporting championship....."
3,Memorial Centre Farmers’ Market,01-Mar-20,NaN,NaN,10:00 AM,"The Memorial Centre Farmers' Market is open year-around and sells an assortment of fresh foods, baked goods, crafts and prepared food from...."
4,Introduction to Encaustic painting,01-Mar-20,Art Noise Studio,https://www.visitkingston.ca/venue/art-noise-studio/,11:00 AM,Enjoy an afternoon of working with wax with Claudia Zilstra. this workshop will introduce you to the art of encaustic painting using Enkaustikos.....
5,Indoor Public Skating at Kingston Memorial Centre,01-Mar-20,NaN,NaN,12:00 PM,Enjoy indoor ice skating with the whole family at Kingston Memorial Centre each Sunday during the winter season from 12 pm to 2 pm. Kingston Memorial Centre is situated...
6,Beers & Bites Tour of Kingston,01-Mar-20,Stone City Ales,https://www.visitkingston.ca/venue/stone-city-ales/,3:00 PM,Spend an afternoon tasting the best of Kingston's world class micro breweries. You and a group of fellow beer lovers will be accompanied by a trained local guide on a walking tour of......
7,Llama Llama – LIVE!,01-Mar-20,The Grand Theatre,https://www.visitkingston.ca/venue/the-grand-theatre/,NaN,"From the beloved book series by Anna Dewdney & the new Netflix series, Llama Llama makes bedtime exciting!"
8,"The Greg Runions Jazz Septet: Something gold, something new at the Isabel",01-Mar-20,Isabel Bader Centre for Performing Arts,https://www.visitkingston.ca/venue/isabel-bader-centre-for-performing-arts/,7:30 PM,"Greg Runions and his all-star Canadian septet present a concert of original and classic jazz music. The ensemble will perform repertoire from the groups 1996 and 2017 recordings plus standards from Duke Ellington's musical partner, the great jazz ....."
9,The Brier Patch Fan Zone presents 6ix Stix,01-Mar-20,The Brier Patch Fan Zone,https://www.visitkingston.ca/venue/the-brier-patch-fan-zone/,10:00 PM,"Rhythm Works 6ix Stix provides a fully interactive show, leaving audiences smiling and happy. After a 40 minute performance, RW gives a motivational..."


In [26]:
df = pd.DataFrame(df['event_description'])
df.columns = ['text']
display(df)

,text
0,Bundle up the little kids and take them to Boucher Park on Clarence Street just around the corner from Springer Market Square and let the kids enjoy the ice slides and ice sculptures. Boucher Park will be open for the whole month of February (weather permitting) in the heart of downtown Kingston ...
1,"Spring Market Square - come and enjoy a lovely, outdoor, skate to music in the historical downtown of Kingston. This public skate is free to the public and makes for a great family outing. Hours of operation are from 8 am to 10 pm Monday to Sunday ...."
2,"IT'S HERE!! The 2020 Tim Hortons Brier, presented by AGI, will be played Feb. 29-March 8 at Leon’s Centre in downtown Kingston.The Tim Hortons Brier is probably the truest of any Canadian sporting championship....."
3,"The Memorial Centre Farmers' Market is open year-around and sells an assortment of fresh foods, baked goods, crafts and prepared food from...."
4,Enjoy an afternoon of working with wax with Claudia Zilstra. this workshop will introduce you to the art of encaustic painting using Enkaustikos.....
5,Enjoy indoor ice skating with the whole family at Kingston Memorial Centre each Sunday during the winter season from 12 pm to 2 pm. Kingston Memorial Centre is situated...
6,Spend an afternoon tasting the best of Kingston's world class micro breweries. You and a group of fellow beer lovers will be accompanied by a trained local guide on a walking tour of......
7,"From the beloved book series by Anna Dewdney & the new Netflix series, Llama Llama makes bedtime exciting!"
8,"Greg Runions and his all-star Canadian septet present a concert of original and classic jazz music. The ensemble will perform repertoire from the groups 1996 and 2017 recordings plus standards from Duke Ellington's musical partner, the great jazz ....."
9,"Rhythm Works 6ix Stix provides a fully interactive show, leaving audiences smiling and happy. After a 40 minute performance, RW gives a motivational..."


In [27]:
def extract_named_ents(text):
    """Extract named entities, and beginning, middle and end idx using spaCy's out-of-the-box model. 
    
    Keyword arguments:
    text -- the actual text source from which to extract entities
    
    """
    return [(ent.text, ent.start_char, ent.end_char, ent.label_) for ent in nlp(text).ents]

def add_named_ents(df):
    """Create new column in data frame with named entity tuple extracted.
    
    Keyword arguments:
    df -- a dataframe object
    
    """
    df['named_ents'] = df['text'].apply(extract_named_ents)    

In [28]:
add_named_ents(df)
display(df)

,text,named_ents
0,Bundle up the little kids and take them to Boucher Park on Clarence Street just around the corner from Springer Market Square and let the kids enjoy the ice slides and ice sculptures. Boucher Park will be open for the whole month of February (weather permitting) in the heart of downtown Kingston ...,"[(Bundle, 0, 6, GPE), (Boucher Park, 43, 55, LOC), (Clarence Street, 59, 74, FAC), (Springer Market Square, 103, 125, FAC), (Boucher Park, 184, 196, FAC), (the whole month of February, 214, 241, DATE), (Kingston, 288, 296, GPE)]"
1,"Spring Market Square - come and enjoy a lovely, outdoor, skate to music in the historical downtown of Kingston. This public skate is free to the public and makes for a great family outing. Hours of operation are from 8 am to 10 pm Monday to Sunday ....","[(Kingston, 102, 110, GPE), (8 am to 10 pm, 217, 230, TIME), (Monday, 231, 237, DATE), (Sunday, 241, 247, DATE)]"
2,"IT'S HERE!! The 2020 Tim Hortons Brier, presented by AGI, will be played Feb. 29-March 8 at Leon’s Centre in downtown Kingston.The Tim Hortons Brier is probably the truest of any Canadian sporting championship.....","[(2020, 16, 20, DATE), (Tim Hortons Brier, 21, 38, PERSON), (AGI, 53, 56, ORG), (Feb. 29-March 8, 73, 88, DATE), (Leon’s Centre, 92, 105, FAC), (Kingston, 118, 126, GPE), (The Tim Hortons Brier, 127, 148, PERSON), (Canadian, 179, 187, NORP)]"
3,"The Memorial Centre Farmers' Market is open year-around and sells an assortment of fresh foods, baked goods, crafts and prepared food from....","[(The Memorial Centre Farmers' Market, 0, 35, ORG)]"
4,Enjoy an afternoon of working with wax with Claudia Zilstra. this workshop will introduce you to the art of encaustic painting using Enkaustikos.....,"[(an afternoon, 6, 18, TIME), (Claudia Zilstra, 44, 59, ORG), (Enkaustikos, 133, 144, ORG)]"
5,Enjoy indoor ice skating with the whole family at Kingston Memorial Centre each Sunday during the winter season from 12 pm to 2 pm. Kingston Memorial Centre is situated...,"[(Kingston Memorial Centre, 50, 74, FAC), (Sunday, 80, 86, DATE), (the winter season, 94, 111, DATE), (12 pm to, 117, 125, QUANTITY), (2 pm, 126, 130, TIME), (Kingston Memorial Centre, 132, 156, FAC)]"
6,Spend an afternoon tasting the best of Kingston's world class micro breweries. You and a group of fellow beer lovers will be accompanied by a trained local guide on a walking tour of......,"[(an afternoon, 6, 18, TIME), (Kingston, 39, 47, GPE)]"
7,"From the beloved book series by Anna Dewdney & the new Netflix series, Llama Llama makes bedtime exciting!","[(Anna Dewdney &, 32, 46, ORG), (Netflix, 55, 62, PERSON), (Llama Llama, 71, 82, PERSON)]"
8,"Greg Runions and his all-star Canadian septet present a concert of original and classic jazz music. The ensemble will perform repertoire from the groups 1996 and 2017 recordings plus standards from Duke Ellington's musical partner, the great jazz .....","[(Greg Runions, 0, 12, PERSON), (Canadian, 30, 38, NORP), (1996, 153, 157, DATE), (2017, 162, 166, DATE), (Duke Ellington's, 198, 214, PERSON)]"
9,"Rhythm Works 6ix Stix provides a fully interactive show, leaving audiences smiling and happy. After a 40 minute performance, RW gives a motivational...","[(Rhythm Works 6ix Stix, 0, 21, PERSON), (40 minute, 102, 111, TIME)]"


In [31]:
column = 'named_ents'
render_entities(180, df, options=options, column=column) # take a look at one of the abstracts

NameError: name 'render_entities' is not defined